In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import lightning as L
from torch.utils.data import DataLoader, Dataset, random_split

import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

from lookup_dict import lookup_dict,id_to_name

In [2]:
root_path=""
dataset_path = os.path.join(root_path,"dataset")
images_dataset_path = os.path.join(dataset_path,"imageNet_images")
eeg_dataset_path = os.path.join(dataset_path,"eeg")

eeg_dataset_name = "eeg_5_95_std.pth"

### TODO

- dataset load
- dataset split
- define model
- add classifier
- testing & accuracy


# Dataset

#### Merge all image folders to create list of images

In [3]:
import random

dir_list = list(os.walk(images_dataset_path))
# skip any unnecessary folders
start_idx = len(dir_list)-40
images_list = []
for sub_dir in dir_list[start_idx:]:
    # images_list+=sub_dir[2]
    images_list.extend(sub_dir[2])
    
images_list = [image_name.replace(".JPEG","") for image_name in images_list]

#### Split images list to 8:1:1

In [4]:
# random.shuffle(images_list)

images_total_size = len(images_list)
train_size = int(images_total_size*0.8)
val_size = int(images_total_size*0.1)
test_size = images_total_size-train_size-val_size

train_images = images_list[:train_size]
val_images = images_list[train_size:train_size+val_size]
test_images = images_list[-1*test_size:]

#### Make train, val, test dataset

In [5]:
class CustomDataset(Dataset):
    def __init__(self,images_list) -> None:
        super().__init__()
        eeg_dataset = torch.load(os.path.join(eeg_dataset_path,eeg_dataset_name))
        self.x_data = []
        self.y_data = []
        # self.y_data = [image_name.split("_")[0] for image_name in images_list]
        
        for eeg_segment in eeg_dataset['dataset']:
            for image_name in images_list:
                if eeg_dataset['images'][eeg_segment['image']] == image_name:
                    self.x_data.append(eeg_segment['eeg'][:,40:480])
                    # all_channel_list = np.array(eeg_segment['eeg'])
                    # self.x_data.append(torch.from_numpy(all_channel_list[:,40:480]))
                    # self.x_data.append(torch.FloatTensor([eeg_sequence[40:480] for eeg_sequence in eeg_segment['eeg']]))
                    class_id = image_name.split("_")[0]
                    self.y_data.append(lookup_dict[class_id])
                    # self.y_data.append(eeg_dataset['labels'][eeg_segment['label']])
                    break
                                    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return len(self.x_data)

In [6]:
b=[1,2,3,4,5]
b[:4]

[1, 2, 3, 4]

In [7]:
a = [[1,2,3],[4,5,6],[7,8,9]]
print(np.array(a)[:2,:3].tolist())
print(a)
print(a[:2])
print([inner[:2] for inner in a])

[[1, 2, 3], [4, 5, 6]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
[[1, 2, 3], [4, 5, 6]]
[[1, 2], [4, 5], [7, 8]]


In [8]:
train_dataset = CustomDataset(train_images)
val_dataset = CustomDataset(val_images)
test_dataset = CustomDataset(test_images)

In [9]:
print(test_dataset[0])
print(test_dataset[0][0].shape)
print(test_dataset[0][0][0].shape)
print(type(test_dataset[0][0][0]))
print(type(test_dataset[0][0]))
print(type(test_dataset[0][1]))

(tensor([[-0.5031, -0.5021, -0.4580,  ...,  0.4509,  0.3556,  0.2762],
        [-0.6303, -0.6110, -0.5451,  ...,  0.1978,  0.0937,  0.0085],
        [ 0.2163,  0.2300,  0.2122,  ...,  0.5433,  0.6652,  0.7695],
        ...,
        [ 0.1370,  0.1335,  0.0989,  ..., -0.0676, -0.1633, -0.2235],
        [-0.0087, -0.0073, -0.0081,  ..., -0.0096, -0.0143, -0.0162],
        [-0.0448, -0.0311, -0.0423,  ..., -0.1442, -0.2242, -0.2662]]), 37)
torch.Size([128, 440])
torch.Size([440])
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'int'>


In [10]:
print(test_dataset[0])
print(test_dataset[0][0].shape)
print(type(test_dataset[0][1]))

(tensor([[-0.5031, -0.5021, -0.4580,  ...,  0.4509,  0.3556,  0.2762],
        [-0.6303, -0.6110, -0.5451,  ...,  0.1978,  0.0937,  0.0085],
        [ 0.2163,  0.2300,  0.2122,  ...,  0.5433,  0.6652,  0.7695],
        ...,
        [ 0.1370,  0.1335,  0.0989,  ..., -0.0676, -0.1633, -0.2235],
        [-0.0087, -0.0073, -0.0081,  ..., -0.0096, -0.0143, -0.0162],
        [-0.0448, -0.0311, -0.0423,  ..., -0.1442, -0.2242, -0.2662]]), 37)
torch.Size([128, 440])
<class 'int'>


In [11]:
# train_loader = DataLoader(train_dataset,shuffle=True,num_workers=1)
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=16)
# val_loader = DataLoader(val_dataset,num_workers=1,persistent_workers=True)
val_loader = DataLoader(val_dataset)
test_loader = DataLoader(test_dataset)

example = next(iter(val_loader))
print(example[0])
print(example[0].shape)
print(example[1])
print(type(example[1]))

tensor([[[-2.2970e-01, -2.0516e-01, -1.4497e-01,  ...,  1.7828e-04,
          -6.1874e-02, -8.4426e-02],
         [-2.3025e-01, -2.3465e-01, -1.9705e-01,  ...,  2.4091e-01,
           1.6397e-01,  9.9622e-02],
         [ 3.7623e-01,  3.9733e-01,  3.5205e-01,  ..., -1.2053e-01,
          -3.9394e-02,  6.2158e-03],
         ...,
         [ 3.5077e-02,  1.9111e-02, -4.2109e-02,  ...,  4.0012e-02,
           5.4250e-02,  4.5684e-02],
         [ 1.7042e-02,  1.5713e-02,  9.6448e-03,  ..., -1.0437e-02,
          -9.6514e-03, -8.0307e-03],
         [-5.6605e-02, -3.8338e-02, -4.0459e-02,  ...,  7.1151e-02,
           6.2956e-02,  5.1401e-02]]])
torch.Size([1, 128, 440])
tensor([14])
<class 'torch.Tensor'>


# Model 만들기

In [12]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [13]:
# with classifier attached
class FeatureExtractorNN(L.LightningModule):
    def __init__(self) -> None:
        super().__init__()
        # self.lstm = nn.LSTM(input_size=128,hidden_size=128,num_layers=128)
        self.lstm = nn.LSTM(input_size=128,hidden_size=128,num_layers=1,batch_first=True)
        self.output = nn.Sequential(
            nn.Linear(in_features=128,out_features=128),
            nn.ReLU()
        )
        self.classifer=nn.Sequential(
            nn.Linear(in_features=128,out_features=40),
            nn.Softmax(dim=1)
        )
        
        self.loss_fn = nn.CrossEntropyLoss()
        self.training_step_outputs = {"correct_num":0,"loss_sum":0}
    
    def forward(self,input):
        # print("\nINPUT")
        # print(input.shape)
        # print(input.shape)
        # input = input.transpose(1,2) if input.dim()==3 else input.transpose(0,1)
        input = input.transpose(1,2)
        # print("INPUT:",input.shape)
        lstm_out, _ = self.lstm(input)
        # tmp_out = lstm_out[:,-1,:] if input.dim()==3 else lstm_out[-1,:]
        tmp_out = lstm_out[:,-1,:]
        out = self.output(tmp_out)
        # print("out shape",out.shape)
        res = self.classifer(out)
        
        return res
    
    def configure_optimizers(self):
        return Adam(self.parameters())
    
    def training_step(self,batch,batch_idx):
        # print("\nbatch:",batch)
        # print("batch shape:",batch[0].shape)
        x,y = batch
        x=x.to(device)
        y=y.to(device)
        # print("\nx:",x)
        # print("x shape:",x.shape)
        # print("\nLABEL")
        # print(x)
        # print(x.shape)
        # print(type(x))
        out = self(x)
        # print(x)
        # print(x[0])
        # print(x.shape)
        # print("\noutput: ",out)
        # print("output shape: ",out.shape)
        # print("\nlabel: ",y)
        loss = self.loss_fn(out,y)
        # print("\nloss: ",loss)
        # print("loss shape: ",loss.shape)
        
        self.log_dict({"train_loss":loss},prog_bar=True,on_epoch=True)
        # print("   ||   training loss:", loss.item())
        preds = out.argmax(dim=1)
        self.training_step_outputs['correct_num']+=(preds==y).sum()
        self.training_step_outputs['loss_sum']+=loss
        # self.training_step_outputs.append(torch.cat((preds==y).sum(),loss),dim=0)
        return loss
    
    def on_train_epoch_end(self) -> None:
        num_correct = self.training_step_outputs['correct_num']
        acc = num_correct/len(train_dataset)
        loss = self.training_step_outputs['loss_sum']/len(train_loader)
        print("EPOCH:",self.current_epoch)
        print("Training accuracy:",acc.item()," ("+str(num_correct.item())+" correct)")
        print("Training loss (average):",loss.item())
        self.training_step_outputs['correct_num']=0
        self.training_step_outputs['loss_sum']=0
    
    def validation_step(self,batch,batch_idx):
        x,y = batch
        x=x.to(device)
        y=y.to(device)
        out = self(x)
        loss = self.loss_fn(out,y)

        self.log_dict({"val_loss":loss},prog_bar=True,on_epoch=True)
        # print("   ||   validation loss:", loss.item())
        return loss
    
    def test_step(self,batch,batch_idx):
        x,y = batch
        
        out = self(x)
        loss = self.loss_fn(out,y)
        
        y_hat = torch.argmax(out,dim=1)
        # print("OUT,YHAT:",out,y_hat)
        test_acc = torch.sum(y == y_hat).item() / (len(y) * 1.0)
        
        self.log_dict({'test_loss': loss, 'test_acc': test_acc},prog_bar=True,on_epoch=True)
        # print("   ||   test loss:",loss.item(), "   ||   test accuracy:",test_acc )


        

In [14]:
a = torch.tensor([[1,2,3,0,0,0],[1,2,0,5,3,4]])
b = torch.tensor([2,3])
(a.argmax(dim=1) == b).sum()

tensor(2)

In [15]:
version_num= 96
epoch = 9
step = 23920
PATH = os.path.join(root_path,"lightning_logs","version_"+str(version_num),"checkpoints","epoch="+str(epoch)+"-step="+str(step)+".ckpt")

model = FeatureExtractorNN()
model.to(device)
# model = FeatureExtractorNN.load_from_checkpoint(PATH)

# outputs = model(train_dataset[0][0])
# print(outputs)

trainer = L.Trainer(max_epochs=10)
# trainer.fit(model,train_dataloaders=train_loader,val_dataloaders=val_loader,ckpt_path=PATH)
# trainer.fit(model,train_dataloaders=train_loader,val_dataloaders=val_loader)
trainer.fit(model,train_dataloaders=train_loader)
# trainer.validate(model, dataloaders=val_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/ms/anaconda3/envs/neuroimagen/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name      | Type             | Params
-----------------------------------------------
0 | lstm      | LSTM             | 132 K 
1 | output    | Sequential       | 16.5 K
2 | classifer | Sequential       | 5.2 K 
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
153 K     Trainable params
0         Non-trainable params
153 K     Total params
0.615     Total estimated model params size (MB)
/Users/ms/anaconda3/envs/neuroimagen/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consi

Training: |          | 0/? [00:00<?, ?it/s]

: 

# Testing Accuracy

In [ ]:
# trainer.test(model,dataloaders=test_loader)

In [ ]:
idx = 6
query = val_dataset[idx][0].unsqueeze(dim=0)
# print(query.shape)
query.to(device)
pred = model(query)
# print(pred)
pred = torch.argmax(pred,dim=1)
# pred = pred.max(dim=1)
# print(pred)
print("predicted: ",id_to_name[lookup_dict[pred.item()]])
print("answer: ",id_to_name[lookup_dict[val_dataset[idx][1]]])


RuntimeError: Input and parameter tensors are not at the same device, found input tensor at cpu and parameter tensor at mps:0

In [ ]:
# Calculate test accuracy
num_correct = 0
for x,y in tqdm(test_loader):
    x=x.to(device)
    y=y.to(device)
    out = model(x)
    y_hat = out.argmax(dim=1)
    # print(y==y_hat)
    num_correct += (y==y_hat).sum()
acc = num_correct/len(test_loader.dataset)

  0%|          | 0/1206 [00:00<?, ?it/s]

tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([False], device='mps:0')
tensor([

KeyboardInterrupt: 

In [ ]:
acc

tensor([0.], device='mps:0')